In [ ]:
input_file = './data/input/traces.csv'
output_file = './data/output/output.csv'
baseline_length = 1500
system_latency = 130

command = f"python3 data_normalization.py {input_file} {output_file} {baseline_length} {system_latency}"
!{command}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


def convert_string_to_array(string):
    return np.fromstring(string.strip("[]"), sep=',')


def build_x_trace(length: int):
    MS_PER_FRAME = 1 / 60 * 1000  # milliseconds per frame
    return np.arange(0, length * MS_PER_FRAME, MS_PER_FRAME)


def plot_median(data, y_column, split_by, y_min=-0.1, y_max=1):
    plt.figure(figsize=(20, 4))
    plt.ylim(y_min, y_max)
    plt.title(y_column)
    plot_df = data.copy()
    plot_df = plot_df.dropna(subset=[y_column])
    plot_df[y_column] = plot_df[y_column].apply(convert_string_to_array)

    max_length = max(plot_df[y_column].apply(len))
    x_trace = build_x_trace(max_length)

    for unique_value, unique_df in plot_df.groupby(split_by):
        if not unique_df.empty:
            padded_arrays = []
            for array in unique_df[y_column]:
                padding = max_length - len(array)
                padded_array = np.pad(array, (0, padding), 'constant', constant_values=np.NaN)
                padded_arrays.append(padded_array)

            label = unique_df['label'].iloc[0]
            stacked = np.stack(padded_arrays)
            median_values = np.nanmedian(stacked, axis=0)
            mean_values = np.nanmean(stacked, axis=0)

            for eye_trace in padded_arrays:
                plt.plot(x_trace, eye_trace, color='lightgray', alpha=0.6)

            plt.plot(x_trace, median_values, label=f'Median {label}', linewidth=2)
            # plt.plot(x_trace, mean_values, label=f'Mean {label}', linewidth=2)

    plt.legend()
    plt.show()

In [ ]:
df = pd.read_csv('./data/output/output.csv')

print(df.columns.tolist())

In [ ]:
plot_median(df, 'eye_lid_trace_leftEyeTrace', 'proto_trial_hash', y_min=-0.1, y_max=0.4)
plot_median(df, 'eye_trace', 'proto_trial_hash', y_min=-0.5, y_max=0.1)
plot_median(df, 'interpolated_trace', 'proto_trial_hash', y_min=-0.5, y_max=0.1)
plot_median(df, 'filtered_trace', 'proto_trial_hash', y_min=-0.5, y_max=0.1)


In [ ]:
plot_median(df, 'first_baseline_corrected_trace', 'proto_trial_hash', y_min=-0.3, y_max=0.5)


In [ ]:
plot_median(df, 'normalized_trace', 'proto_trial_hash', y_min=-0.3, y_max=1.2)

In [ ]:
plot_median(df, 'baseline_trace', 'proto_trial_hash', y_min=-0.3, y_max=1.2)

In [ ]:
df.info()

In [ ]:
df.head(100)